In [1]:
from flask import Flask, render_template
from flask_cors import CORS

import json
import os
import time

In [2]:
app = Flask(__name__)

CORS(app)

In [3]:
cache_file_name = 'cache.json'
max_cache_size = 50

def read_cache():
    cache = {'string': {}, 'hashtag': {}, 'user': {}, 'author': {}, 'retweet': {}, 'metric':{} }
    
    if os.path.exists(cache_file_name):
#         try:
        with open(cache_file_name, 'r') as file:
            cache = json.load(file)
#         except:
#             cache = {'string': {}, 'hashtag': {}, 'user': {}, 'author': {}, 'retweet': {}, 'metric':{} }
    return cache
        
def write_cache(cache):
    with open(cache_file_name, 'w') as file:  
        json.dump(cache, file)

def clear_cache():
    cache = {'string': {}, 'hashtag': {}, 'user': {}, 'author': {}, 'retweet': {}, 'metric':{} }
    if os.path.exists(cache_file_name):
        os.remove(cache_file_name)
    return cache

def cache_print():
    for key in cache.keys():
        print(key)
        print(list(cache[key].keys()))
        print()


def get_from_cache(cache_type, word):
    if word in cache[cache_type].keys():
        return True, cache[cache_type][word]
    return False, None

def add_to_cache(cache_type, word, value):
    if len(cache[cache_type]) >= max_cache_size:
        cache[cache_type].pop( next(iter(cache[cache_type])) )
    cache[cache_type][word] = value
    write_cache(cache)

In [4]:
def time_filter(data, stime, etime, fetch_time):
    if stime=="na" and etime=="na":
        return {'data': data, 'fetch_time': fetch_time }
    
    if stime=="na":
        res = []
        etime = int(etime)/1000
        for tweet in data:
            if tweet['created_at'] <= etime:
                res.append(tweet)
        return {'data': res, 'fetch_time': fetch_time }

    if etime=="na":
        res = []
        stime = int(stime)/1000
        for tweet in data:
            if tweet['created_at'] >= stime:
                res.append(tweet)
        return {'data': res, 'fetch_time': fetch_time }
    
    res = []
    stime, etime = int(stime)/1000, int(etime)/1000
    for tweet in data:
        if stime <= tweet['created_at'] <= etime:
            res.append(tweet)
    return {'data': res, 'fetch_time': fetch_time }

In [5]:
col_keys = [ 'id', 'name', 'screen_name', 'location', 'url', 'description', 'verified','followers_count',
            'friends_count', 'listed_count', 'favourites_count', 'statuses_count', 'created_at' ]

def make_dict(item):
    kv = {}
    for i in range(len(col_keys)):
        kv[col_keys[i]] = item[i]
    return kv
    
def make_dict_list(lst):
    res = []
    for item in lst:
        res.append(make_dict(item))  
    return res

t_user = [ 'name', 'count' ]
def make_user_dict_list(lst):
    res = []
    for item in lst:
        kv = {}
        for i in range(len(t_user)):
            kv[t_user[i]] = item[i]
        res.append(kv)  
    return res

def extract_val(key, lst):
    res = []
    for item in lst:
        res.append(item[key])
    return res

In [6]:
import mysql.connector as msql

sql_password = "password"

def getUserByWord(word):
    conn = msql.connect(host='localhost', database='TwitterDB', user='root', password=sql_password)
    cursor = conn.cursor()
    query = f"SELECT * FROM users_data WHERE name LIKE '%{word}%' or screen_name LIKE '%{word}%';"
    cursor.execute(query)
    a=cursor.fetchall()
    cursor.close()
    conn.close()
    return make_dict_list(a)

def getUserByID(word):
    conn = msql.connect(host='localhost', database='TwitterDB', user='root', password=sql_password)
    cursor = conn.cursor()
    query = f"SELECT * FROM users_data WHERE id = '{word}';"
    cursor.execute(query)
    a=cursor.fetchall()
    cursor.close()
    conn.close()
    return make_dict(a[0]) if len(a)>0 else {}

def get_top_users():
    conn = msql.connect(host='localhost', database='TwitterDB', user='root', password=sql_password)
    cursor = conn.cursor()
    query = f"SELECT name, followers_count FROM users_data ORDER BY followers_count DESC LIMIT 10 ;"
    cursor.execute(query)
    a=cursor.fetchall()
    cursor.close()
    return make_user_dict_list(a)

In [7]:
from pymongo import MongoClient, DESCENDING

drop_field = {'_id': 0}

def getTweetbyID(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"id": word}
    result = collection.find(query, drop_field).sort("score", DESCENDING)
    a=[]
    for i in result:
        a.append(i)
        
    return a[0] if len(a)>0 else {}

def getTweetsbyString(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"text": {"$regex": word, "$options": "i"} }
    result = collection.find(query, drop_field).sort("score", DESCENDING)
    a=[]
    for i in result:
        a.append(i)  
    return a

def getTweetsbyHashtag(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"hashtags": {"$regex": f"{word}", "$options": "i"}}
    result = collection.find(query, drop_field).sort("score", DESCENDING)
    a=[]
    for i in result:
        a.append(i) 
    return a

def getTweetsbyUserIDsList(lst):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"user.id": {"$in": lst}}
    result = collection.find(query, drop_field).sort("score", DESCENDING)
    a=[]
    for i in result:
        a.append(i)
    return a

def getTweetsbyUserID(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]

    query = {"user.id": word}
    result = collection.find(query, drop_field).sort("score", DESCENDING)
    a=[]
    for i in result:
        a.append(i)
    return a

def getRetweetsbyUserID(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Retweets"]

    query = {"user.id": word}
    result = collection.find(query, drop_field).sort("score", DESCENDING)
    a=[]
    for i in result:
        a.append(i)
    return a

def getRetweetsByOriginalTweetID(word):
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Retweets"]

    query = {"retweeted_status.id": word}
    result = collection.find(query, drop_field).sort("score", DESCENDING)
    a=[]
    for i in result:
        a.append(i)
    return a


def get_top_tweets():
    client = MongoClient()
    db = client["TwitterDB"]
    collection = db["Tweets"]
    
    query = {}
    result = collection.find(query, drop_field).sort("score", DESCENDING).limit(10)
    a=[]
    for i in result:
        a.append(i)
    return a

In [8]:
@app.route("/")
def index():
    return render_template('index.html')


@app.route("/string/<word>/<stime>/<etime>", methods=["GET"])
def string_tweet(word, stime, etime):
    start_time = time.time()
    inCache, res = get_from_cache('string', word)
    if inCache:
        print("time taken:", time.time() - start_time)
        return json.dumps(time_filter(res, stime, etime, time.time() - start_time))
    
    res = getTweetsbyString(word)
    
    add_to_cache('string', word, res)
    print("time taken:", time.time() - start_time)
    return json.dumps(time_filter(res, stime, etime, time.time() - start_time))

@app.route("/hashtag/<word>/<stime>/<etime>", methods=["GET"])
def hashtag_tweet(word, stime, etime):
    start_time = time.time()
    inCache, res = get_from_cache('hashtag', word)
    if inCache:
        return json.dumps(time_filter(res, stime, etime, time.time() - start_time))
        
    res = getTweetsbyHashtag(word)
    
    add_to_cache('hashtag', word, res)
    return json.dumps(time_filter(res, stime, etime, time.time() - start_time))

@app.route("/user/<word>/<stime>/<etime>", methods=["GET"])
def user_tweet(word, stime, etime):
    start_time = time.time()
    inCache, res = get_from_cache('user', word)
    if inCache:
        return json.dumps(time_filter(res, stime, etime, time.time() - start_time))
        
    usr_lst = getUserByWord(word)
    id_lst = extract_val('id', usr_lst)
    res = getTweetsbyUserIDsList(id_lst)
    
    add_to_cache('user', word, res)
    return json.dumps(time_filter(res, stime, etime, time.time() - start_time))


@app.route("/author/<id>", methods=["GET"])
def get_author(id):
    start_time = time.time()
    inCache, res = get_from_cache('author', id)
    if inCache:
        return json.dumps({'data': res, 'fetch_time': time.time() - start_time })
    
    res = getUserByID(id)
    res['tweets'] = getTweetsbyUserID(id)
    res['retweets'] = getRetweetsbyUserID(id)
    
    add_to_cache('author', id, res)
    return json.dumps({'data': res, 'fetch_time': time.time() - start_time })

@app.route("/retweet/<id>", methods=["GET"])
def get_retweets(id):
    start_time = time.time()
    inCache, res = get_from_cache('retweet', id)
    if inCache:
        return json.dumps({'data': res, 'fetch_time': time.time() - start_time })
    
    res = getTweetbyID(id)
    res['retweets'] = getRetweetsByOriginalTweetID(id)
    
    add_to_cache('retweet', id, res)
    return json.dumps({'data': res, 'fetch_time': time.time() - start_time })


@app.route("/metric", methods=["GET"])
def get_metric():
    start_time = time.time()
    inCache, res = get_from_cache('metric', 'all')
    if inCache:
        return json.dumps({'data': res, 'fetch_time': time.time() - start_time })

    users = get_top_users()
    tweets = get_top_tweets()
#     users = u_temp
#     tweets = t_temp
    
    res = [users, tweets]
    
    add_to_cache('metric', 'all', res)
    return json.dumps({'data': res, 'fetch_time': time.time() - start_time })

In [9]:
cache = read_cache()

In [10]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Apr/2023 15:56:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 15:56:30] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [28/Apr/2023 15:56:30] "GET /static/script.js HTTP/1.1" 304 -
127.0.0.1 - - [28/Apr/2023 15:56:33] "GET /metric HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2023 15:56:33] "GET /static/verified.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Apr/2023 15:56:39] "GET /metric HTTP/1.1" 200 -
